# Workshop n°2: Parametric (spatial) building frame in reinforced concrete

# Roberto Nunziato, matricola: 438778

<h2>Descrizione</h2>

Sfruttando il progetto del workshop n°1 l'obiettivo è quello di realizzare una struttura in cemento armato, realizzata  prendendo spunto da questo modello di riferimento:

![alt text](modelloDisegno.png)


<h2>Librerie importate</h2>

In [1]:
from pyplasm import *
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.005652 seconds


<h2>Funzione che disegna un telaio piano</h2>

In [2]:
"""Input: beamSection,pillarSection,distPillar,interstoryHeight"""

def draw(beamSection,pillarSection,distPillar,interstoryHeight):
    px = pillarSection[0]
    py = pillarSection[1]
    bx = beamSection[0]
    bz = beamSection[1]
    xPillar = []
    yPillar = []
    zPillar = []
    xBeam = []
    yBeam = []
    zBeam = [] 
    distance = 0  #distance between pillars
    """Build pillars"""
    for i in distPillar:
        xPillar = [px]
        yPillar = yPillar + [py,-i]
        distance = distance + i + py   
    yPillar = yPillar + [py]

    """Build beams"""
    for i in interstoryHeight:
        zPillar = zPillar + [i,-bz] 
        xBeam = [bx]
        yBeam = [distance+py]
        zBeam = zBeam + [-i,bz]

    pilTemp = PROD([QUOTE(xPillar),QUOTE(yPillar)])
    pillar = PROD([pilTemp,QUOTE(zPillar)])
    traTemp = PROD([QUOTE(xBeam),QUOTE(yBeam)])
    beam = PROD([traTemp,QUOTE(zBeam)])

    model = STRUCT([pillar,beam])
    return model


<h2>Funzione che costruisce le travi orizzontali</h2>

In [3]:
def drawOrizzontalBeam(bx,bz,px,py,distPillar,interstoryHeight):
    xBeamOrizz = [bx]
    yBeamOrizz = []
    zBeamOrizz = []
    
    for i in interstoryHeight:
        zBeamOrizz = zBeamOrizz + [-i,bz]
    for i in distPillar:
        yBeamOrizz = yBeamOrizz + [py,-i]
    yBeamOrizz = yBeamOrizz + [py]
    traOrizzTemp = PROD([QUOTE(xBeamOrizz),QUOTE(yBeamOrizz)])
    beamOrizz = PROD([traOrizzTemp,QUOTE(zBeamOrizz)])

    model = STRUCT([beamOrizz])
    return model


<h2>Funzione che legge dal file .csv e costruisce la struttura finale</h2>

L'input della funzione è rappresentato da un file .csv, in cui:

-Nelle linee dispari ci sono i vettori di traslazione tridimensionali

-Nelle linee pari ci sono i parametri di riferimento del telaio piano   (beamSection,pillarSection,distPillar,interstoryHeight)

In [4]:
def ggpl_bone_structure(fileName):
    x = []
    y = []
    z = []
    frames = []
    transactions = []
    orizzontalBeam = []
    structure = []
    structure2 = []
    Vx=0
    
    with open(fileName,'rU') as csvfile:
        reader = csv.DictReader(csvfile)
        cont = 2 #contatore righe
        for row in reader:
            if(cont%2 != 0): #riga dispari
                beamSection = row['x'].replace("(","").replace(")","").split(";")
                pillarSection = row['y'].replace("(","").replace(")","").split(";")
                beamSectionX = eval(beamSection[0])
                beamSectionZ = eval(beamSection[1])
                pillarSectionX = eval(pillarSection[0])
                pillarSectionY = eval(pillarSection[1])
                                         
                distPillar = []
                for elem in row['z'].replace("[","").replace("]","").split(";"):
                    distPillar.append(eval(elem))
                
                interstoryHeight = []
                for elem in row['k'].replace("[","").replace("]","").split(";"):
                    interstoryHeight.append(eval(elem))
                
                orizzontalBeam.append(drawOrizzontalBeam(Vx,beamSectionZ,pillarSectionX,pillarSectionY,distPillar,interstoryHeight))                                                             
                frames.append(draw((beamSectionX,beamSectionZ),(pillarSectionX,pillarSectionY),distPillar,interstoryHeight))
                
            if(cont%2 == 0): #riga pari
                Vx = x = eval(row['x'])
                y = eval(row['y'])
                z = eval(row['z'])
                transactions.append(T([1,2,3])([x,y,z]))
               
            cont = cont+1
            
    i = 0
    for elem in frames:
        structure.append(transactions[i])
        structure.append(frames[i])
        if(i+1 < len(frames)):
            structure2.append(transactions[i])
            structure2.append(orizzontalBeam[i+1])
        i = i+1
        
    view1 = STRUCT(structure)
    view2 = STRUCT(structure2)
    VIEW(STRUCT([view1,view2]))
    

In [5]:
ggpl_bone_structure("frame_data_438778.csv")

<h2>Risultato ottenuto: </h2>


![alt text](strutturaFinale.png)